# 01 Data preprocessing
Haetaan tiedot

In [ ]:
from ucimlrepo import fetch_ucirepo
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

chronic_kidney_disease = fetch_ucirepo(id=336)

df = chronic_kidney_disease.data.original
df.replace('nan', np.nan, inplace=True)
df


Poistetaan turhat sarakkeet ja nimetään uudestaan lyhenteet käytetyistä sarakkeista


In [ ]:
df = df.drop(columns=['rbc', 'pc', 'pcc', 'ba', 'sc', 'htn', 'dm', 'cad', 'appet', 'pe', 'ane'])
df.rename(columns={'bp':'Blood pressure', 'sg':'Specific gravity', 'al':'Albumin', 'su':'Sugar', 'bgr':'Blood glucose random', 'bu':'Blood urea', 'sod':'Sodium', 'pot':'Potassium', 'hemo':'Hemoglobin', 'pcv':'Packed cell volume', 'wbcc':'White blood cell count', 'rbcc':'Red blood cell count'}, inplace=True)
df

Muutetaan hemoglobiini g/dl -> g/l

In [ ]:
df['Hemoglobin'] = df['Hemoglobin'] * 10
df['Hemoglobin']

Muutetaan luokka sarakkeen arvot

In [ ]:

df['class'] = df['class'].map({'ckd': 'a', 'notckd': 'c'})
len(df)

Poistetaan rivit, joissa on 3 tai enemmän tyhjiä arvoja

In [ ]:
print('orginal lenght', len(df))
df = df[df.isna().sum(axis = 1) < 3]
print('new length', len(df))
df

# Dataframes

In [ ]:
affected = df[df['class'] == 'a']
control = df[df['class'] == 'c']
control

In [ ]:
affected

In [ ]:
print('affected rows:', len(affected), 'control rows:', len(control))

# Basic statistics

In [ ]:
affected.describe()

In [ ]:
control.describe()

# Histogram

Affected

In [ ]:
def plot_data(df):
    plt.figure(figsize=(20, 15))
    for i, col in enumerate(df.select_dtypes(include=['int', 'float']).columns):
        plt.subplot(3,5, i + 1)
        df[col].plot(kind='hist', title=col)
        plt.xlabel(col)
plot_data(affected)

Control

In [ ]:
plot_data(control)

affected taulussa potassium arvo näyttää olevan väärin merkitty kummankin taulun perusteella

In [ ]:
print('Max potassium in affected:', affected['Potassium'].max(), 'Max potassium in control:', control['Potassium'].max())

In [ ]:
affected = affected[affected['Potassium'] < 20]

Myös Affected White blood cell count maksimi on todella suuri verrattuna normaaliin

In [ ]:
print('Max white blood cell count in affected:', affected['White blood cell count'].max(), 'Max white blood cell count in control:', control['White blood cell count'].max())

In [ ]:
affected = affected[affected['White blood cell count'] < 230000]

Sama verenpaineen suhteen

In [ ]:
print('Max blood pressure in affected:', affected['Blood pressure'].max(), 'Max blood pressure in control:', control['Blood pressure'].max())

In [ ]:
affected = df[df['Blood pressure'] < 180]

In [ ]:
affected['Potassium'].unique()

In [ ]:
plot_data(affected)

In [ ]:
plot_data(control)

# Correlation matrix

In [ ]:
affected_correlation = affected.corr(numeric_only=True)
control_correlation = control.corr(numeric_only=True)
affected_correlation

In [ ]:
control_correlation

# Heatmap

In [ ]:
def heatmap(correlation_matrix):
    plt.figure(figsize=(10, 10))
    plt.imshow(correlation_matrix, cmap='coolwarm', interpolation='none')
    plt.colorbar()
    plt.xticks(range(len(correlation_matrix)), correlation_matrix.columns, rotation=90)
    plt.yticks(range(len(correlation_matrix)), correlation_matrix.columns)
    plt.show()

Affected


In [ ]:
heatmap(affected_correlation)

Control


In [ ]:
heatmap(control_correlation)